In [1]:
import os

In [2]:
from pathlib import Path

In [3]:
import numpy as np

In [4]:
import pandas as pd

In [5]:
from sklearn.pipeline import Pipeline

In [6]:
from sklearn.compose import ColumnTransformer

In [7]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [8]:
from sklearn.impute import SimpleImputer

In [9]:
import joblib

In [10]:
RANDOM_STATE = 42

In [11]:
PROJECT_ROOT = Path('.')

In [12]:
SPLITS_DIR = PROJECT_ROOT / 'data' / 'splits'

In [13]:
ARTIFACTS_DIR = PROJECT_ROOT / 'artifacts'

In [14]:
FEATURES_DIR = PROJECT_ROOT / 'data' / 'features'

In [15]:
ARTIFACTS_DIR.mkdir(parents=True,exist_ok=True)

In [16]:
FEATURES_DIR.mkdir(parents=True, exist_ok=True)

In [17]:
train_path = SPLITS_DIR / 'train.csv'

In [18]:
valid_path = SPLITS_DIR / 'valid.csv'

In [19]:
test_path = SPLITS_DIR / 'test.csv'

In [20]:
train_df = pd.read_csv(train_path)

In [21]:
valid_df = pd.read_csv(valid_path)

In [22]:
test_df = pd.read_csv(test_path)

In [23]:
train_df.shape, valid_df.shape, test_df.shape

((8543, 18), (2563, 18), (1099, 18))

In [24]:
train_df.head()

,administrative,administrative_duration,informational,informational_duration,product_related,product_related_duration,bounce_rates,exit_rates,page_values,special_day,month,operating_systems,browser,region,traffic_type,visitor_type,weekend,revenue
0,5,18.000000,1,9.5,17,414.333333,0.000000,0.005263,0.0,0.0,May,2,2,2,6,Returning_Visitor,0,0
1,0,0.000000,0,0.0,11,1055.500000,0.018182,0.042424,0.0,0.0,May,2,2,8,18,Returning_Visitor,0,0
2,0,0.000000,0,0.0,8,607.000000,0.037500,0.062500,0.0,0.0,Mar,1,1,1,3,Returning_Visitor,0,0
3,6,200.333333,0,0.0,9,154.000000,0.000000,0.030769,0.0,0.0,May,1,1,1,1,Returning_Visitor,0,0
4,2,55.000000,1,36.0,24,914.506061,0.015385,0.016667,0.0,0.4,May,3,2,8,3,Returning_Visitor,0,0


In [25]:
TARGET_COL = 'revenue'

In [26]:
def split_xy(df:pd.DataFrame, target_col:str = TARGET_COL):
    X = df.drop(columns = [target_col])
    y = df[target_col].astype(int)
    return X,y

In [27]:
X_train_raw, y_train = split_xy(train_df)

In [28]:
X_valid_raw, y_valid = split_xy(valid_df)

In [29]:
X_test_raw, y_test = split_xy(test_df)

In [30]:
y_train.value_counts(normalize=True)

revenue
0    0.843615
1    0.156385
Name: proportion, dtype: float64

In [31]:
EPS = 1e-9

In [32]:
def make_features(df:pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    def safe_div(numer,denom):
        return numer / (denom + EPS)

    df['total_pages'] = (
        df['administrative'] + df['informational'] + df['product_related']
    )
    df['total_duration'] = (
        df['administrative_duration'] + df['informational_duration'] + df['product_related_duration'] 
    )

    df['product_page_share'] = safe_div(df['product_related'], df['total_pages']).clip(0,1)
    df['product_duration_share'] = safe_div(df['product_related_duration'], df['total_duration']).clip(0,1)
    df['avg_time_per_page'] = safe_div(df['total_duration'], df['total_pages']).clip(0,5000)
    df['admin_to_product_ratio'] = safe_div(df['administrative'], df['product_related']).clip(0,10)
    df['info_to_product_ratio'] = safe_div(df['informational'], df['product_related']).clip(0,10)

    df['exit_minus_bounce'] = df['exit_rates'] - df['bounce_rates'].clip(-1,1)
    df['bounce_x_exit'] = df['bounce_rates'] * df['exit_rates'].clip(0,1)

    for col in [
        "administrative_duration",
        "informational_duration",
        "product_related_duration",
        "total_duration",
        "total_pages",
    ]:
        df[f"log1p_{col}"] = np.log1p(df[col].clip(lower = 0))

    return df

    
    

In [33]:
LEAKY_COLS = ['page_values']

In [34]:
def build_feature_sets(X_raw:pd.DataFrame):
    X_fe = make_features(X_raw)
    X_safe = X_fe.drop(columns = [c for c in LEAKY_COLS if c in X_fe.columns],errors = 'ignore')
    X_full = X_fe.copy()
    return X_safe, X_full

In [35]:
X_train_safe, X_train_full = build_feature_sets(X_train_raw)

In [36]:
X_valid_safe, X_valid_full = build_feature_sets(X_valid_raw)

In [37]:
X_test_safe, X_test_full = build_feature_sets(X_test_raw)

In [38]:
assert list(X_train_safe.columns) == list(X_valid_safe.columns) == list(X_test_safe.columns)

In [39]:
assert list(X_train_full.columns) == list(X_valid_full.columns) == list(X_test_full.columns)

In [40]:
print('Safe:', X_train_safe.shape, 'FULL:', X_train_full.shape)

Safe: (8543, 30) FULL: (8543, 31)


In [41]:
feature_rationale = [
    ("total_pages", "numeric", "Sayfa gezinme yoğunluğu satın alma niyeti göstergesi", "↑ purchase propensity", "low"),
    ("total_duration", "numeric", "Toplam süre niyeti gösterir; çarpık dağılım log ile desteklenir", "↑ purchase propensity", "low"),
    ("product_page_share", "numeric", "Gezinmenin ne kadarı ürün odaklı?", "↑ intent signal", "low"),
    ("product_duration_share", "numeric", "Sürenin ne kadarı ürün odaklı?", "↑ intent signal", "low"),
    ("avg_time_per_page", "numeric", "Sayfa başı ortalama süre", "↑ engagement", "low"),
    ("admin_to_product_ratio", "numeric", "Admin vs ürün odaklı gezinme dengesi", "behavior signal", "low"),
    ("info_to_product_ratio", "numeric", "Bilgi vs ürün odaklı gezinme dengesi", "behavior signal", "low"),
    ("exit_minus_bounce", "numeric", "Exit ve bounce farkı davranış sinyali", "behavior signal", "low"),
    ("bounce_x_exit", "numeric", "Hızlı çıkış davranışı", "behavior signal", "low"),
    ("log1p_*", "numeric", "Sağa çarpıklığı azaltır, model stabilitesini artırır", "stability", "low"),
    ("page_values", "numeric", "Çok güçlü ama leakage riski olabilir", "powerful predictor", "medium/high"),
]


In [42]:
rationale_df = pd.DataFrame(feature_rationale, columns=["feature", "type", "why", "expected_effect", "risk"])

In [43]:
rationale_df

,feature,type,why,expected_effect,risk
0,total_pages,numeric,Sayfa gezinme yoğunluğu satın alma niyeti göst...,↑ purchase propensity,low
1,total_duration,numeric,Toplam süre niyeti gösterir; çarpık dağılım lo...,↑ purchase propensity,low
2,product_page_share,numeric,Gezinmenin ne kadarı ürün odaklı?,↑ intent signal,low
3,product_duration_share,numeric,Sürenin ne kadarı ürün odaklı?,↑ intent signal,low
4,avg_time_per_page,numeric,Sayfa başı ortalama süre,↑ engagement,low
5,admin_to_product_ratio,numeric,Admin vs ürün odaklı gezinme dengesi,behavior signal,low
6,info_to_product_ratio,numeric,Bilgi vs ürün odaklı gezinme dengesi,behavior signal,low
7,exit_minus_bounce,numeric,Exit ve bounce farkı davranış sinyali,behavior signal,low
8,bounce_x_exit,numeric,Hızlı çıkış davranışı,behavior signal,low
9,log1p_*,numeric,"Sağa çarpıklığı azaltır, model stabilitesini a...",stability,low


In [44]:
rationale_df.to_csv(FEATURES_DIR / "feature_rationale_day24.csv", index=False)


In [45]:
def build_preprocess_pipeline(X:pd.DataFrame):
    cat_cols = X.select_dtypes(include = ['object']).columns.tolist()
    num_cols = [c for c in X.columns if c not in cat_cols]

    numeric_transformer = Pipeline(steps = [
        ('imputer', SimpleImputer(strategy = 'median')),
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps = [
        ('imputer', SimpleImputer(strategy = 'most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown = 'ignore'))
    ])

    preprocessor = ColumnTransformer(
        transformers = [
            ('num', numeric_transformer, num_cols),
            ('cat', categorical_transformer, cat_cols),
        ],
        remainder = 'drop'
    )

    return preprocessor, num_cols, cat_cols

In [46]:
preprocessor_safe, num_cols_safe, cat_cols_safe = build_preprocess_pipeline(X_train_safe)

In [47]:
print('num_cols:', len(num_cols_safe))

num_cols: 28


In [48]:
print('cat_cols:', cat_cols_safe)

cat_cols: ['month', 'visitor_type']


In [49]:
preprocessor_safe.fit(X_train_safe)

,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers contains sparse matrices,these will be stacked as a sparse matrix if the overall density islower than this value. Use ``sparse_threshold=0`` to always returndense. When the transformed output consists of all dense data, thestacked result will be dense, and this keyword will be ignored.",0.3
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.",None
,"transformer_weights transformer_weights: dict, default=NoneMultiplicative weights for features per transformer. The output of thetransformer is multiplied by these weights. Keys are transformer names,values the weights.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each transformer will beprinted as it is completed.",False
,"verbose_feature_names_out verbose_feature_names_out: bool, str or Callable[[str, str], str], default=True- If True, :meth:`ColumnTransformer.get_feature_names_out` will prefix all feature names with the name of the transformer that generated that feature. It is equivalent to setting `verbose_feature_names_out=""{transformer_name}__{feature_name}""`.- If False, :meth:`ColumnTransformer.get_feature_names_out` will not prefix any feature names and will error if feature names are not unique.- If ``Callable[[str, str], str]``, :meth:`ColumnTransformer.get_feature_names_out` will rename all the features using the name of the transformer. The first argument of the callable is the transformer name and the second argument is the feature name. The returned string will be the new feature name.- If ``str``, it must be a string ready for formatting. The given string will be formatted using two field names: ``transformer_name`` and ``feature_name``. e.g. `

In [50]:
artifact = {
    'preprocessor': preprocessor_safe,
    'feature_set': 'SAFE',
    'leaky_cols_dropped': [c for c in LEAKY_COLS if c in X_train_full.columns],
    'num_cols': num_cols_safe,
    'cat_cols': cat_cols_safe,
    
}

In [51]:
joblib.dump(artifact, ARTIFACTS_DIR / 'preprocess_day24_safe.joblib')

['artifacts\\preprocess_day24_safe.joblib']

In [52]:
def save_xy(X:pd.DataFrame, y:pd.Series, prefix:str):
    out = X.copy()
    out[TARGET_COL] = y.values
    out.to_csv(FEATURES_DIR / f"{prefix}.csv",index=False)

In [53]:
save_xy(X_train_safe, y_train, 'train_safe_fe')

In [54]:
save_xy(X_valid_safe,y_valid, 'valid_safe_fe')

In [55]:
save_xy(X_test_safe,y_test, 'test_safe_fe')

In [56]:
save_xy(X_train_full, y_train, 'train_full_fe')

In [57]:
save_xy(X_valid_full, y_valid, 'valid_full_fe')

In [58]:
save_xy(X_test_full, y_test, 'test_full_fe')